In [1]:
# python
import pickle
import numpy as np
import random
from scipy.sparse import csr_matrix

# AutDEC
from autdec.bb_cln import *
from autdec.perm_utils import perm_mat_from_aut
from autdec.igraph_auts import *

## Bivariate Bicycle Codes: Circuit-Level Noise Simulations

### Choose BB code:

In [2]:
n = 72

For the [[72,12,6]], [[90,8,10]] and [[144,12,12]] BB codes, the graph automorphism files of the detector error model check matrix are pregenerated and stored in the `bivariate_bicycle_codes\graph_auts` folder. 

For the rest of the BB codes in the original paper, the detector error model graph automorphisms can be computed below by entering the corresponding physical qubit number `n`.

In [3]:
# Load auts
if n == 72: 
    code_name = 'BB72'
    chk_shape = (252,2232)
    k = 12
    with open(f'graph_auts/n{n}k{k}_graph_autsZ.pkl', 'rb') as f:
        auts = pickle.load(f)
        auts_ord = len(auts)
    DEM_col_perms = [np.eye(chk_shape[1],dtype=int)]
    DEM_row_perms = [np.eye(chk_shape[0],dtype=int)]
    for a in auts: 
        P = perm_mat_from_aut(a,chk_shape[0]+chk_shape[1])
        DEM_col_perms.append(P[:chk_shape[1],:chk_shape[1]])
        DEM_row_perms.append(P[chk_shape[1]:,chk_shape[1]:])
elif n==90:
    code_name = 'BB90'
    chk_shape = (495,4590)
    k = 8
    with open(f'graph_auts/n{n}k{k}_graph_autsZ.pkl', 'rb') as f:
        auts = pickle.load(f)
        auts_ord = len(auts)
    DEM_col_perms = [np.eye(chk_shape[1],dtype=int)]
    DEM_row_perms = [np.eye(chk_shape[0],dtype=int)]
    for a in auts: 
        P = perm_mat_from_aut(a,chk_shape[0]+chk_shape[1])
        DEM_col_perms.append(P[:chk_shape[1],:chk_shape[1]])
        DEM_row_perms.append(P[chk_shape[1]:,chk_shape[1]:])
elif n==144:
    code_name = 'BB144'
    chk_shape = (936,8784)
    k = 12
    with open(f'graph_auts/n{n}k{k}_graph_autsZ.pkl', 'rb') as f:
        auts = pickle.load(f)
        auts_ord = len(auts)
    DEM_col_perms = [np.eye(chk_shape[1],dtype=int)]
    DEM_row_perms = [np.eye(chk_shape[0],dtype=int)]
    for a in auts: 
        P = perm_mat_from_aut(a,chk_shape[0]+chk_shape[1])
        DEM_col_perms.append(P[:chk_shape[1],:chk_shape[1]])
        DEM_row_perms.append(P[chk_shape[1]:,chk_shape[1]:])

else: 
    code_name = f'BB{n}'
    chk = bb_dem_matrix(code_name) # DEM check matrix
    chk_shape = chk.shape
    DEM_col_perms, DEM_row_perms = vertex_graph_auts_from_bliss(chk,print_order=False)
    auts_ord=len(DEM_row_perms)
print(f"Automorphisms: {auts_ord+1}")


Automorphisms: 36


## Choose ensemble size

In [4]:
# Pick random elements from list of automorphisms
batchsize = 18
auts_ind_list=random.sample(range(1,auts_ord),batchsize)
auts_ind_list=np.hstack((0,auts_ind_list)) # always include identity perm (base decoder)


DEM_col_perms = [DEM_col_perms[a] for a in auts_ind_list]
DEM_row_perms = [DEM_row_perms[a] for a in auts_ind_list]
print(f'No of random auts taken: {len(auts_ind_list)}')

No of random auts taken: 19


# AutDEC

In [5]:
e = 0.0050
num_shots = 10**3
BP_iters = 1000
BP_method = "ms"
ms_factor = 1.0
BP_hyperparams = {'max_iter': BP_iters,
                  'bp_method': BP_method,
                  'ms_scaling_factor': ms_factor}


print(code_name)


print(autdecode_bb_cln(code_name,e, num_shots, DEM_col_perms, DEM_row_perms, base_decoder='BP',decoder_hyperparams=BP_hyperparams,basis='Z'))

BB72
Starting decoding.
Trial no      BaseDec Errors   AutDEC Errors
100                       10               5
200                       20              14
300                       30              23
400                       44              33
500                       51              39
600                       66              51
700                       82              64
800                       96              73
900                      109              83
1000                     122              92
(1000, np.int64(122), np.int64(92))
